In [1]:
import pandas as pd

In [64]:
df = pd.read_csv('../../../dades/raposos/certificats-ine-raposos.csv')

/var/folders/0b/9qg1xh5d46j8phx5051jbncw0000gn/T/ipykernel_25649/3490750228.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../../dades/raposos/certificats-ine-raposos.csv')


In [68]:
df.columns

Index(['nom_provincia', 'codi_provincia', 'referencia_cadastral',
       'zona_climatica', 'any_construccio', 'us_edifici',
       'energia_primaria_no_renovable', 'consum_energia_final',
       'cost_anual_aproximat_energia', 'vehicle_electric', 'solar_termica',
       'solar_fotovoltaica', 'sistema_biomassa', 'xarxa_districte',
       'energia_geotermica', 'valor_aillaments', 'valor_finestres',
       'energia_calefaccio', 'energia_refrigeracio', 'energia_acs',
       'energia_enllumenament', 'ventilacio_us_residencial',
       'rehabilitacio_energetica', 'actuacions_rehabilitacio', 'data_entrada',
       'mundissec', 'any', 'codi_districte', 'codi_postal', 'poblacio', 'year',
       'renta_neta_mitja_por_persona', 'renta_neta_mitja_por_hogar',
       'renta_mitja_por_unitat_consum', 'renta_mediana_por_unitat_consum',
       'renta_bruta_mitja_por_persona', 'renta_bruta_mitja_por_hogar',
       'renta_bruta_mitja_por_persona_fuente_salario',
       'renta_bruta_mitja_por_persona_fuen

In [69]:
df['comarca']

KeyError: 'comarca'

In [54]:
# Transformar data_entrada
df_processed = df
df_processed['data_entrada'] = pd.to_datetime(df['data_entrada'])

# Filtrar 'us_habitatge' distinto de 'Terciari'
df_processed = df_processed[df_processed['us_edifici'] != 'Terciari']

# Filtrar las filas donde 'referencia_cadastral' aparece al menos 2 veces en todo el dataset
df_processed = df_processed[df_processed.groupby('referencia_cadastral')['referencia_cadastral'].transform('count') >= 2]

In [ ]:
# formula: reducció consum / cost
# [{
#       "referencia_cadastral": "",
#       "coste_inversion_rehabilitacion": "",
#       "ahorro": "",
#       "ahorro/inversion": "",
#       "per cada edifici --> eficiencia energetica, renta, gini": "",
#       "provincia": "",
#       "min_year": "2016",
#       "min_cost": 6000,
#       "min_consum": 6000,
#       "max_year": "2016",
#       "max_cost": 6000,
#       "max_consum": 6000,
# }]

In [55]:
res_eei = {}
for index, row in df_processed.iterrows():
    if not row['referencia_cadastral'] in res_eei:
        res_eei[row['referencia_cadastral']] = {
            "referencia_cadastral": row['referencia_cadastral'],
            "min_year": None,
            "min_cost": None,
            "min_consum": None,
            "max_year": None,
            "max_cost": None,
            "max_consum": None,
        }
    
    if res_eei[row['referencia_cadastral']]['min_year'] is None or res_eei[row['referencia_cadastral']]['min_year'] > row['data_entrada'].year:
        res_eei[row['referencia_cadastral']]['min_year'] = row['data_entrada'].year
        res_eei[row['referencia_cadastral']]['min_cost'] = row['cost_anual_aproximat_energia']
        res_eei[row['referencia_cadastral']]['min_consum'] = row['consum_energia_final']
    elif res_eei[row['referencia_cadastral']]['max_year'] is None or res_eei[row['referencia_cadastral']]['max_year'] < row['data_entrada'].year:
        res_eei[row['referencia_cadastral']]['max_year'] = row['data_entrada'].year
        res_eei[row['referencia_cadastral']]['max_cost'] = row['cost_anual_aproximat_energia']
        res_eei[row['referencia_cadastral']]['max_consum'] = row['consum_energia_final']


In [56]:
res_eei

{'8566002DF0986N0001QA': {'referencia_cadastral': '8566002DF0986N0001QA',
  'min_year': 2020,
  'min_cost': 41.89,
  'min_consum': 264.14,
  'max_year': 2023,
  'max_cost': 8.4,
  'max_consum': 134.73},
 '8569601DF0986N0006DH': {'referencia_cadastral': '8569601DF0986N0006DH',
  'min_year': 2015,
  'min_cost': 0.0,
  'min_consum': 0.0,
  'max_year': 2015,
  'max_cost': 1144.65,
  'max_consum': 316.6},
 '8567220DF0986N0035OD': {'referencia_cadastral': '8567220DF0986N0035OD',
  'min_year': 2015,
  'min_cost': 83.72,
  'min_consum': 17.8,
  'max_year': None,
  'max_cost': None,
  'max_consum': None},
 '8566314DF0986N0001WA': {'referencia_cadastral': '8566314DF0986N0001WA',
  'min_year': 2015,
  'min_cost': 0.0,
  'min_consum': 0.0,
  'max_year': 2022,
  'max_cost': 11.93,
  'max_consum': 195.7},
 '8566003DF0986N0001PA': {'referencia_cadastral': '8566003DF0986N0001PA',
  'min_year': 2017,
  'min_cost': 3343.49,
  'min_consum': 230.47,
  'max_year': None,
  'max_cost': None,
  'max_consum': 

In [60]:
df_eei = pd.DataFrame(res_eei.values())

# Filtrem
# df_eei = df_eei[(df_eei['min_year'] != df_eei['max_year']) & df_eei['max_year'].notna() & (df_eei['min_cost'] > 0)]

df_eei

,referencia_cadastral,min_year,min_cost,min_consum,max_year,max_cost,max_consum
0,8566002DF0986N0001QA,2020,41.89,264.14,2023.0,8.40,134.73
1,8569601DF0986N0006DH,2015,0.00,0.00,2015.0,1144.65,316.60
2,8567220DF0986N0035OD,2015,83.72,17.80,NaN,NaN,NaN
3,8566314DF0986N0001WA,2015,0.00,0.00,2022.0,11.93,195.70
4,8566003DF0986N0001PA,2017,3343.49,230.47,NaN,NaN,NaN
...,...,...,...,...,...,...,...
88749,6629101CG2762N0037WQ,2020,6.29,93.64,NaN,NaN,NaN
88750,2803517DG3420S0001OA,2024,31.01,416.63,2024.0,1.41,8.89
88751,5801120EG0550S0001XT,2025,21.45,286.93,2025.0,20.01,265.99
88752,7134825CF2573C0001ZO,2025,10.58,93.43,2025.0,11.25,86.38


In [ ]:
# Datos de Idealista
df_idealista_provincia